In [1]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Download covariate data over sampled locations

This notebook downloads a set of covariate data from sampled locations

The point collection is assumed to contain the field `first` (referring to the ecoregion).

## Import API and get credentials

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the API into your session.

In [2]:
import ee

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [3]:
# Import Google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# Trigger the GEE authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=9yhBRyfoC2xoAyY83_nCt1T0ikXtJOvaopIR3TfEDmU&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g6nGTOSFurAz5x_81RSmbTGN5hN5ZrP8TsglfZcjH1Z5zAXLXZK56Q

Successfully saved authorization token.


### User inputs

Enter the following inputs:
- `start_date`: start date of the period of interest
- `end_date`: end date of the period of interest
- `output_name`: name of the output file
- `points_collection`: name of the collection with sampled points
- `init_point`: initial point to be processed
- `end_point`: final point to be processed (setting this to `None` equals the last point in the collection)

In [5]:
# Inputs
start_date = ee.Date('1995-01-01')# start date of period of interest
end_date = ee.Date('2020-01-01')# end date of period of interest
init_point = 0# start processing this point onwards
end_point = 10#None# last point to be processed, if None: this equals the latest point
# name of GEE collection of points - from each of these points we want to extract a time series
points_collection='users/wandadekeersmaecker/RETURN/samples_forest_loss_fire_n1000_VAL'
output_name= '/content/gdrive/MyDrive/RETURN/Landsat_NBR/samples_forest_fire_nat_n1000_VAL_' + str(init_point) + '_' + str(end_point) + '.csv'#  path of output file


### List sampled points

Get a list of the sampled points 

In [6]:
# number of points in the collection
npontos=ee.FeatureCollection(points_collection).size().getInfo()

if end_point == None:
	end_point=npontos
print(end_point)
# GEE list of points
lista_pontos = ee.FeatureCollection(points_collection).toList(npontos)


10


## Data extraction
For each point, a vector with the following information is added as one line to the output file:
 - `j`: ID of point
 - `ecoreg`: ecoregion 
 - `point.geometry().coordinates().getInfo()`: coordinate
 - `cec_Vals`: soil CEC
 - `nitrogen_Vals`: soil nitrogen
 - `soc_Vals`: soil soc
 - `silt_Vals`: soil silt conent
 - `clay_Vals`: soil clay content
 - `cwd_Vals`: cumulative climatic water deficit
 - `elev_Vals`: elevation
 - `slope_Vals`: slope
 - `aspect_Vals`: aspect
 - `tcmean500_Vals`: mean tree cover in 500m square kernel
 - `tcmean1000_Vals`: mean tree cover in 1000m square kernel
 - `tcmean5000_Vals`: mean tree cover in 5000m square kernel
 - `tcsd500_Vals`: sd tree cover in 500m square kernel
 - `tcsd1000_Vals`: sd tree cover in 1000m square kernel
 - `tcsd5000_Vals`: sd tree cover in 5000m square kernel
 - `distWat_Vals`: distance to open water
 - `dist_Vals`: distance to no forest area

In [7]:
# iterate over the points and write surface reflectance data to a file
with open(output_name,"a") as f:
	for j in range(init_point,end_point):
		print(j)
		# get one point 
		point = ee.Feature(lista_pontos.get(j))
		# CEC
		cec = ee.Image("projects/soilgrids-isric/cec_mean")
		cec_obj=cec.reduceRegion('mean',point.geometry(),30)
		cec_List=cec_obj.getInfo()
		cec_Vals=list(cec_List.values())# CEC values
		# N
		nitrogen = ee.Image("projects/soilgrids-isric/nitrogen_mean")
		nitrogen_obj=nitrogen.reduceRegion('mean',point.geometry(),30)
		nitrogen_List=nitrogen_obj.getInfo()
		nitrogen_Vals=list(nitrogen_List.values())# nitrogen values
		# SOC
		soc = ee.Image("projects/soilgrids-isric/soc_mean")
		soc_obj=soc.reduceRegion('mean',point.geometry(),30)
		soc_List=soc_obj.getInfo()
		soc_Vals=list(soc_List.values())# soc values
		# silt
		silt = ee.Image("projects/soilgrids-isric/silt_mean")
		silt_obj=silt.reduceRegion('mean',point.geometry(),30)
		silt_List=silt_obj.getInfo()
		silt_Vals=list(silt_List.values())# silt values
		# clay
		clay = ee.Image("projects/soilgrids-isric/clay_mean")
		clay_obj=clay.reduceRegion('mean',point.geometry(),30)
		clay_List=clay_obj.getInfo()
		clay_Vals=list(clay_List.values())# clay values
    # CCWD
		cwd = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").select('def').filterDate(start_date,end_date).filterBounds(point.geometry()).sum()
		cwd_obj=ee.Image(cwd).reduceRegion('mean',point.geometry(),30)
		cwd_List=cwd_obj.getInfo()
		cwd_Vals=list(cwd_List.values())# cwd values
    # elevation
		elev = ee.Image("USGS/SRTMGL1_003")
		elev_obj=elev.reduceRegion('mean',point.geometry(),30)
		elev_List=elev_obj.getInfo()
		elev_Vals=list(elev_List.values())# elev values
    # slope 
		slope = ee.Terrain.slope(ee.Image("USGS/SRTMGL1_003"))
		slope_obj=slope.reduceRegion('mean',point.geometry(),30)
		slope_List=slope_obj.getInfo()
		slope_Vals=list(slope_List.values())# slope values
    # aspect
		aspect = ee.Terrain.aspect(ee.Image("USGS/SRTMGL1_003"))
		aspect_obj=aspect.reduceRegion('mean',point.geometry(),30)
		aspect_List=aspect_obj.getInfo()
		aspect_Vals=list(aspect_List.values())# aspect values
    # treecover
		han = ee.Image("UMD/hansen/global_forest_change_2019_v1_7")
		tcmean500 = han.select('treecover2000').reduceNeighborhood(ee.Reducer.mean(), ee.Kernel.square(500,'meters'))
		tcmean500_obj=tcmean500.reduceRegion('mean',point.geometry(),30)
		tcmean500_List=tcmean500_obj.getInfo()
		tcmean500_Vals=list(tcmean500_List.values())# mean tree cover in 500m square kernel
		tcmean1000 = han.select('treecover2000').reduceNeighborhood(ee.Reducer.mean(), ee.Kernel.square(1000,'meters'))
		tcmean1000_obj=tcmean1000.reduceRegion('mean',point.geometry(),30)
		tcmean1000_List=tcmean1000_obj.getInfo()
		tcmean1000_Vals=list(tcmean1000_List.values())# mean tree cover in 1000m square kernel
		tcmean5000 = han.select('treecover2000').reduceNeighborhood(ee.Reducer.mean(), ee.Kernel.square(5000,'meters'))
		tcmean5000_obj=tcmean5000.reduceRegion('mean',point.geometry(),30)
		tcmean5000_List=tcmean5000_obj.getInfo()
		tcmean5000_Vals=list(tcmean5000_List.values())# # mean tree cover in 5000m square kernel
		tcsd500 = han.select('treecover2000').reduceNeighborhood(ee.Reducer.stdDev(), ee.Kernel.square(500,'meters'))
		tcsd500_obj=tcsd500.reduceRegion('mean',point.geometry(),30)
		tcsd500_List=tcsd500_obj.getInfo()
		tcsd500_Vals=list(tcsd500_List.values())# sd tree cover in 500m square kernel
		tcsd1000 = han.select('treecover2000').reduceNeighborhood(ee.Reducer.stdDev(), ee.Kernel.square(1000,'meters'))
		tcsd1000_obj=tcsd1000.reduceRegion('mean',point.geometry(),30)
		tcsd1000_List=tcsd1000_obj.getInfo()
		tcsd1000_Vals=list(tcsd1000_List.values())# sd tree cover in 1000m square kernel
		tcsd5000 = han.select('treecover2000').reduceNeighborhood(ee.Reducer.stdDev(), ee.Kernel.square(5000,'meters')) 
		tcsd5000_obj=tcsd5000.reduceRegion('mean',point.geometry(),30)
		tcsd5000_List=tcsd5000_obj.getInfo()
		tcsd5000_Vals=list(tcsd5000_List.values())# sd tree cover in 5000m square kernel
		# Distance to water
		wat = ee.Image("JRC/GSW1_2/GlobalSurfaceWater")
		water = wat.select('max_extent').eq(1)
		projectionWat = water.select(0).projection()
		scaleWat = 30
		distWat = water.fastDistanceTransform(4000).sqrt().reproject(crs=projectionWat,scale=scaleWat).multiply(ee.Image.pixelArea().sqrt())
		distWat_obj=distWat.reduceRegion('mean',point.geometry(),30)
		distWat_List=distWat_obj.getInfo()
		distWat_Vals=list(distWat_List.values())# distance to water
		# Distance to buildings
		#buildup = ee.Image("JRC/GHSL/P2016/BUILT_LDSMT_GLOBE_V1")
		#buildings = buildup.select('built').gt(2)
		#projectionBU = buildup.select(0).projection()
		#scaleBU = 38.21851414258853
		#distBU = buildings.fastDistanceTransform(5000).sqrt().reproject(crs=projectionBU,scale=scaleBU).multiply(ee.Image.pixelArea().sqrt());
		#distBU_obj=distBU.reduceRegion('mean',point.geometry(),scaleBU)
		#distBU_List=distBU_obj.getInfo()
		#distBU_Vals=list(distBU_List.values())# CEC values
		# Distance to no-forest area
		han = ee.Image("UMD/hansen/global_forest_change_2019_v1_7")
		For = han.select('treecover2000').lt(60)
		projection = For.select(0).projection();
		scale = 30
		dist = For.fastDistanceTransform(4000).sqrt().reproject(crs=projection,scale=scale).multiply(ee.Image.pixelArea().sqrt());
		dist_obj=dist.reduceRegion('mean',point.geometry(),30)
		dist_List=dist_obj.getInfo()
		dist_Vals=list(dist_List.values())# distance to no-forest area
    # ecoregion of the point
		ecoreg = point.get('ecoReg').getInfo()
		id = point.get('id').getInfo()
    # vector to be exported
		write_vector=[id,ecoreg,point.geometry().coordinates().getInfo(),cec_Vals,nitrogen_Vals,soc_Vals,silt_Vals,clay_Vals,cwd_Vals, elev_Vals,slope_Vals,aspect_Vals,tcmean500_Vals,tcmean1000_Vals,tcmean5000_Vals,tcsd500_Vals,tcsd1000_Vals,tcsd5000_Vals,distWat_Vals,dist_Vals]#,distBU_Vals,dist_Vals
		f.write(str(write_vector)+'\n')
		f.flush()

0
1
2
3
4
5
6
7
8
9
